In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

In [2]:
df = pd.read_csv("absent.csv")
df.head()

,Reason for absence,Month of absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Service time,Age,Workloadavg,Hit target,Disciplinary failure,Education,Son,Social drinker,Social smoker,Pet,Weight,Height,Body mass index,Absenteeism time in hours
0,26,7,3,1,289,36,13,33,239.0,97,0,1,2,1,0,1,90,172,30,4
1,0,7,3,1,118,13,18,50,239.0,97,1,1,1,1,0,0,98,178,31,0
2,23,7,4,1,179,51,18,38,239.0,97,0,1,0,1,0,0,89,170,31,2
3,7,7,5,1,279,5,14,39,239.0,97,0,1,2,1,1,0,68,168,24,4
4,23,7,5,1,289,36,13,33,239.0,97,0,1,2,1,0,1,90,172,30,2


In [3]:
df.shape

(740, 20)

In [4]:
df.dtypes

Reason for absence                   int64
Month of absence                     int64
Day of the week                      int64
Seasons                              int64
Transportation expense               int64
Distance from Residence to Work      int64
Service time                         int64
Age                                  int64
 Workloadavg                       float64
Hit target                           int64
Disciplinary failure                 int64
Education                            int64
Son                                  int64
Social drinker                       int64
Social smoker                        int64
Pet                                  int64
Weight                               int64
Height                               int64
Body mass index                      int64
Absenteeism time in hours            int64
dtype: object

In [5]:
df.isnull()

,Reason for absence,Month of absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Service time,Age,Workloadavg,Hit target,Disciplinary failure,Education,Son,Social drinker,Social smoker,Pet,Weight,Height,Body mass index,Absenteeism time in hours
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
735,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
736,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
737,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
738,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [6]:
df.isnull().sum()

Reason for absence                 0
Month of absence                   0
Day of the week                    0
Seasons                            0
Transportation expense             0
Distance from Residence to Work    0
Service time                       0
Age                                0
 Workloadavg                       0
Hit target                         0
Disciplinary failure               0
Education                          0
Son                                0
Social drinker                     0
Social smoker                      0
Pet                                0
Weight                             0
Height                             0
Body mass index                    0
Absenteeism time in hours          0
dtype: int64

In [7]:
dataset = df.values # convert into 2D array
print(dataset)

[[ 26.   7.   3. ... 172.  30.   4.]
 [  0.   7.   3. ... 178.  31.   0.]
 [ 23.   7.   4. ... 170.  31.   2.]
 ...
 [  0.   0.   3. ... 170.  34.   0.]
 [  0.   0.   4. ... 170.  35.   0.]
 [  0.   0.   6. ... 175.  25.   0.]]


In [8]:
m = dataset[:,:-1] 
n = dataset[:,-1] # target column
num_of_columns = m.shape[1] # number of columns
print(num_of_columns)
num_of_cols_seven_five = int((75*num_of_columns)/100)# take only 75% columns from total columns
print(num_of_cols_seven_five)

19
14


In [9]:
numberFold = 5 
labels = np.unique(n) # unique labels in target column
numOfLabels = len(labels) # number of unique labels 
print(numOfLabels)

19


In [10]:
def kfold(dataset):
    classify = list()
    rowsPerFold = list()
    for unik in labels: # iterate all unique labels
        rows = dataset[np.where(n==unik)] # take "unik" label data 
        np.random.shuffle(rows) # shuffle rows
        classify.append(rows) 
        l = len(rows)
        rowsPerFold.append(int(l/numberFold)) # calculating the percent of labels in every fold
    return classify,rowsPerFold

In [11]:
def stratified(dataset):
    classify, rowsPerFold = kfold(dataset)
    datasetPartition = list()
    a = 0
    # maintain the percent of labels in every fold
    while(a<numberFold): # iterate on number of fold
        fold = list()
        b = 0
        while(b<numOfLabels): # iterate to fold on percent of labels
            if(a==numberFold-1):
                x = classify[b][p:]
                fold.extend(x) 
            else:
                p = rowsPerFold[b] * a
                q = rowsPerFold[b] * (a+1)                
                x = classify[b][p:q]
                fold.extend(x)
            b = b + 1
        np.random.shuffle(fold)
        datasetPartition.append(fold)
        a = a + 1
    return datasetPartition

In [12]:
validateData = stratified(dataset) # data with stratification
print(validateData)

[[array([ 19.,   4.,   3.,   3., 289.,  36.,  13.,  33., 326.,  96.,   0.,
         1.,   2.,   1.,   0.,   1.,  90., 172.,  30.,   8.]), array([ 23.,   9.,   3.,   1., 184.,  42.,   7.,  27., 247.,  92.,   0.,
         1.,   0.,   0.,   0.,   0.,  58., 167.,  21.,   2.]), array([ 19.,   6.,   6.,   3., 233.,  51.,   1.,  31., 275.,  96.,   0.,
         2.,   1.,   1.,   0.,   8.,  68., 178.,  21.,   8.]), array([ 19.,   7.,   2.,   1., 289.,  36.,  13.,  33., 275.,  98.,   0.,
         1.,   2.,   1.,   0.,   1.,  90., 172.,  30.,  32.]), array([ 28.,  11.,   3.,   4., 225.,  26.,   9.,  28., 306.,  93.,   0.,
         1.,   1.,   0.,   0.,   2.,  69., 169.,  24.,   3.]), array([ 23.,  12.,   6.,   4., 225.,  26.,   9.,  28., 280.,  98.,   0.,
         1.,   1.,   0.,   0.,   2.,  69., 169.,  24.,   3.]), array([ 22.,  10.,   2.,   4., 369.,  17.,  12.,  31., 265.,  88.,   0.,
         1.,   3.,   1.,   0.,   0.,  70., 169.,  25.,   8.]), array([ 14.,  12.,   5.,   4., 361.,  52.,   3

In [13]:
def firstFeatureSelect(x_train, x_test, y_train, y_test,i):    
    accuracy = list() # accuracy of every features
    j = 0
    while(j<numberFold):
        xtr = x_train[:,i].reshape(-1,1) 
        xts = x_test[:,i].reshape(-1,1)
        classifier = RandomForestClassifier(n_estimators = 22, criterion = 'entropy', random_state = 41) # random forest classifier using information gain
        classifier.fit(xtr, y_train)
        classifierscore = classifier.score(xts, y_test) # accuracy of random forest on test data
        accuracy.append(classifierscore)
        j = j + 1
    avg = np.mean(accuracy) # take average of all accuracies
    return avg   

In [14]:
def remainingFeatureSelect(attributes,x_train, x_test, y_train, y_test):
    score = list()
    for i in range(num_of_columns): # iterate to remaining features and add with the existing features
        if(i in attributes): # skip the feature if it already exist in list and score set to zero
            score.append(0)
            continue
        precision = list()
        attributes_copy = attributes.copy()
        attributes_copy.append(i)
        j = 0
        while(j<numberFold):
            xtr = x_train[:,attributes_copy]
            xts = x_test[:,attributes_copy]
            classifier = RandomForestClassifier()
            classifier.fit(xtr, y_train)
            classifierscore = classifier.score(xts, y_test)
            precision.append(classifierscore)
            j = j + 1
        score.append(np.mean(precision))
    arghigh = np.argmax(score) # get index of highest score of the feature
    maximum = score[arghigh] # score of highest max feature
    return maximum, arghigh

In [15]:
def sequentialForwardSelection(partition_of_data):
    i = 0
    # set of train and test with the 5th fold of data
    while(i<numberFold):
        test_data = np.asarray(partition_of_data[i])
        x_test = test_data[:,:-1]   
        y_test = test_data[:,-1]
        array = np.arange(numberFold).tolist()
        array.remove(i)
        train_data = partition_of_data[array[0]]
        array_length = len(array)
        for a in range(1,array_length):
            tup = (train_data, partition_of_data[array[a]])
            train_data = np.concatenate(tup)
        x_train = train_data[:,:-1]
        y_train = train_data[:,-1]
        i = i + 1

    attributes = list() # list stores the features 
    score = list() # store the score of the features
    while(i<num_of_columns):
        res = firstFeatureSelect(x_train, x_test, y_train, y_test,i) # call the firstfeatureselection fuction
        score.append(res)
        i = i + 1
    attributes.append(np.argmax(score)) # take column number of highest accuracy
    
    precision_scores = list() # list stores the accuracy of all selected features
    while(len(attributes) < num_of_cols_seven_five):   # iterate to 75% features
        res2, res3 = remainingFeatureSelect(attributes,x_train, x_test, y_train, y_test) # call the reamainingfeatureselect function
        precision_scores.append(res2)
        attributes.append(res3)
    indices = np.argmax(precision_scores) # get index of highest score of the feature
    final_score = precision_scores[indices] # score of highest max feature
    best_indexes = attributes[:indices+1]
    return best_indexes,final_score

In [16]:
final_cols, score = sequentialForwardSelection(validateData)
print("The Precision is: ", score)
print("The Best Features is: ", final_cols)

The Precision is:  0.8337837837837838
The Best Features is:  [0, 8, 16, 2, 3]
